In [1]:
import anndata
import xarray as xr
import pandas as pd
import scipy.sparse as ss
import h5py

In [14]:
use_genes1 = pd.read_csv('Seurat_HVGs.txt', index_col=0).index
use_genes1 = use_genes1.str.replace('-', '_')
use_genes2 = pd.read_csv('SCF_hvgs.txt', index_col=0).index

use_genes = use_genes1 | use_genes2
use_genes.name = 'gene'

In [16]:
mch_meta = pd.read_csv('human_frontal_mch_metadata.tsv.gz', sep='\t', index_col=0)
mch_meta['cell_id'] = mch_meta.index.str[:-4]

rna_meta = pd.read_csv('human_frontal_rna_metadata.tsv.gz', sep='\t', index_col=0)
rna_meta['cell_id'] = mch_meta.index.str[:-4]

## RNA

In [18]:
mcds = xr.open_mfdataset('/home/hanliu/project/human_multiomic/mCTN/dataset/snmCT_*.withRNA.mcds', 
                         concat_dim='cell', combine='nested').squeeze()
rna_data = mcds['rna_da'].load().sel(cell=rna_meta['cell_id'].values, gene=use_genes).to_pandas()

In [20]:
rna_adata = anndata.AnnData(X=rna_data.values,
                            obs=rna_meta,
                            var=pd.DataFrame([], index=rna_data.columns))
rna_adata.write_h5ad('rna_adata.h5ad', compression=True)
rna_adata

... storing 'Technology' as categorical
... storing 'Brain Region' as categorical
... storing 'AD Index' as categorical
... storing 'ClusterAnno' as categorical
... storing 'MajorType' as categorical
... storing 'SubMarker' as categorical
... storing 'SubClusterAnno' as categorical
... storing 'MajorMarker' as categorical
... storing 'Individual ID' as categorical
... storing 'Individual Label' as categorical
... storing 'm3c_cluster' as categorical


AnnData object with n_obs × n_vars = 3898 × 6876
    obs: 'Technology', 'Brain Region', 'AD Index', 'Total Methylome Reads', 'Overall Adjusted mCG%', 'Overall Adjusted mCH%', 'Overall mCCC%', 'tsne_0', 'tsne_1', 'umap_0', 'umap_1', 'pca_0', 'pca_1', 'pca_2', 'pca_3', 'MajorCluster', 'ClusterAnno', 'MajorType', 'SubClusterInternalID', 'subumap_0', 'subumap_1', 'subtsne_0', 'subtsne_1', 'SubMarker', 'SubClusterAnno', 'MajorMarker', 'Individual ID', 'Individual Label', 'Individual Age', 'm3c_cluster', 'cell_id'

## mC Frac

In [26]:
mcds = xr.open_mfdataset(
    '/home/hanliu/project/human_multiomic/mCTN/dataset/ForSeurat/gene_frac/snmCT_*.withRNA.mcdsgene_da_rate.nc',
    concat_dim='cell',
    combine='nested').squeeze()
mch_data = mcds.sel(mc_type='HCHN')['gene_da_rate'].load().sel(
    cell=mch_meta['cell_id'].values, gene=mcds.get_index('gene') & use_genes).to_pandas()

In [27]:
mch_adata = anndata.AnnData(X=mch_data.values,
                            obs=mch_meta,
                            var=pd.DataFrame([], index=mch_data.columns))
mch_adata.write_h5ad('mch_adata.h5ad', compression=True)
mch_adata

... storing 'Technology' as categorical
... storing 'Brain Region' as categorical
... storing 'AD Index' as categorical
... storing 'ClusterAnno' as categorical
... storing 'MajorType' as categorical
... storing 'SubMarker' as categorical
... storing 'SubClusterAnno' as categorical
... storing 'MajorMarker' as categorical
... storing 'Individual ID' as categorical
... storing 'Individual Label' as categorical
... storing 'm3c_cluster' as categorical


AnnData object with n_obs × n_vars = 3898 × 6771
    obs: 'Technology', 'Brain Region', 'AD Index', 'Total Methylome Reads', 'Overall Adjusted mCG%', 'Overall Adjusted mCH%', 'Overall mCCC%', 'tsne_0', 'tsne_1', 'umap_0', 'umap_1', 'pca_0', 'pca_1', 'pca_2', 'pca_3', 'MajorCluster', 'ClusterAnno', 'MajorType', 'SubClusterInternalID', 'subumap_0', 'subumap_1', 'subtsne_0', 'subtsne_1', 'SubMarker', 'SubClusterAnno', 'MajorMarker', 'Individual ID', 'Individual Label', 'Individual Age', 'm3c_cluster', 'cell_id'

## Prepare LIGER

In [7]:
use_genes = pd.read_csv('../input_archive/human_frontal_mch_hvfeatures.reverse.tsv.gz', 
                        index_col=0, sep='\t').index
use_genes.name = 'gene'
use_genes

Index(['ENSG00000001084.11_4', 'ENSG00000001460.17_3', 'ENSG00000001461.16_2',
       'ENSG00000001626.14_3', 'ENSG00000001629.9_3', 'ENSG00000002587.9_3',
       'ENSG00000002746.14_3', 'ENSG00000002822.15_3', 'ENSG00000003096.14_4',
       'ENSG00000003137.8_2',
       ...
       'ENSG00000285090.1_1', 'ENSG00000285095.1_1', 'ENSG00000285283.1_1',
       'ENSG00000285382.1_1', 'ENSG00000285404.1_1', 'ENSG00000285446.1_1',
       'ENSG00000285454.1_1', 'ENSG00000285467.1_1', 'ENSG00000285476.1_1',
       'ENSG00000285509.1_1'],
      dtype='object', name='gene', length=5107)

In [8]:
rna_adata = anndata.read_h5ad('rna_adata.h5ad')
mc_adata = anndata.read_h5ad('mch_adata.h5ad')

In [9]:
use_genes = use_genes & rna_adata.var_names & mc_adata.var_names
use_genes

Index(['ENSG00000001084.11_4', 'ENSG00000001460.17_3', 'ENSG00000001461.16_2',
       'ENSG00000001626.14_3', 'ENSG00000001629.9_3', 'ENSG00000002587.9_3',
       'ENSG00000002746.14_3', 'ENSG00000002822.15_3', 'ENSG00000003096.14_4',
       'ENSG00000003137.8_2',
       ...
       'ENSG00000285090.1_1', 'ENSG00000285095.1_1', 'ENSG00000285283.1_1',
       'ENSG00000285382.1_1', 'ENSG00000285404.1_1', 'ENSG00000285446.1_1',
       'ENSG00000285454.1_1', 'ENSG00000285467.1_1', 'ENSG00000285476.1_1',
       'ENSG00000285509.1_1'],
      dtype='object', name='gene', length=5107)

In [15]:
rna_adata = rna_adata[:, use_genes].copy()
csr_x = ss.csr_matrix(rna_adata.X)
with h5py.File('rna.h5', 'w') as f:
    f['matrix/barcodes'] = rna_adata.obs_names.values
    f['matrix/data'] = csr_x.data
    f['matrix/indices'] = csr_x.indices
    f['matrix/indptr'] = csr_x.indptr
    f['matrix/features/name'] = rna_adata.var_names.values

In [16]:
mc_adata = mc_adata[:, use_genes].copy()
mc_adata.X = mc_adata.X.max() - mc_adata.X

csr_x = ss.csr_matrix(mc_adata.X)
with h5py.File('mch.h5', 'w') as f:
    f['matrix/barcodes'] = mc_adata.obs_names.values
    f['matrix/data'] = csr_x.data
    f['matrix/indices'] = csr_x.indices
    f['matrix/indptr'] = csr_x.indptr
    f['matrix/features/name'] = mc_adata.var_names.values